In [7]:
from utils.fs import SeamlessInteractionFS
import shutil
import os
import glob

/opt/hpcaas/.mounts/fs-0df31b178aa4037ac/home/yaoj/projects/seamless_interaction/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [90]:
batch_size_dict = {
    "video": 300,
    "audio": 500,
    "metadata": 1500,
    "smplh": 1000,
    "movement": 1000,
    "boxes_and_keypoints": 1000,
}

In [39]:
label, split = "improvised", "test"
fs = SeamlessInteractionFS()

In [43]:
fs._local_dir

'/checkpoint/seamless/yaoj/data/seamless_interaction/dataset/'

In [45]:
files = glob.glob(f"{fs._local_dir}/*/*/video/*")

In [46]:
len(files)

131070

In [4]:
!ls /checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/dev/audio

batch_0000  batch_0001


In [6]:
!tree -L 4 /checkpoint/seamless/yaoj/data/batched_seamless_interaction/

/checkpoint/seamless/yaoj/data/batched_seamless_interaction/
├── improvised
│   ├── dev
│   │   ├── audio
│   │   │   ├── batch_0000
│   │   │   └── batch_0001
│   │   └── video
│   │       ├── batch_0000
│   │       ├── batch_0001
│   │       ├── batch_0002
│   │       └── batch_0003
│   ├── test
│   │   ├── audio
│   │   │   ├── batch_0000
│   │   │   ├── batch_0001
│   │   │   ├── batch_0002
│   │   │   ├── batch_0003
│   │   │   └── batch_0004
│   │   └── video
│   │       ├── batch_0000
│   │       ├── batch_0001
│   │       ├── batch_0002
│   │       ├── batch_0003
│   │       ├── batch_0004
│   │       ├── batch_0005
│   │       ├── batch_0006
│   │       ├── batch_0007
│   │       ├── batch_0008
│   │       └── batch_0009
│   └── train
│       └── metadata
│           ├── batch_0000
│           ├── batch_0001
│           ├── batch_0002
│           ├── batch_0003
│           ├── batch_0004
│           ├── batch_0005
│           ├── batch_0006
│           ├── batch_0007
│        

In [82]:
!du -h -d 1 /checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/dev/video

35G	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/dev/video/batch_0002
9.7G	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/dev/video/batch_0001
38G	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/dev/video/batch_0003
6.8G	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/dev/video/batch_0000
89G	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/dev/video


In [ ]:
label, split = "improvised", "train"
feature = "video"
all_files = fs._fetch_filelist(label=label, split=split)
partitions = fs.partition_filelist(all_files, partition_by="vendor-session-interaction")
batches = fs.create_batches(partitions, batch_size=batch_size_dict[feature])
for i in range(len(batches)):
    dest_fpath = f"/checkpoint/seamless/yaoj/data/batched_seamless_interaction/{label}/{split}/{feature}/batch_{i:04d}/"
    os.makedirs(dest_fpath, exist_ok=True)
    for file_id in batches[i]:
        src_fpath = f"{fs._local_dir}/{label}/{split}/{feature}/{file_id}.mp4"
        shutil.move(src_fpath, dest_fpath)

In [78]:
feature = "audio"
batches = fs.create_batches(partitions, batch_size=batch_size_dict[feature])
for i in range(len(batches)):
    dest_fpath = f"/checkpoint/seamless/yaoj/data/batched_seamless_interaction/{label}/{split}/{feature}/batch_{i:04d}/"
    os.makedirs(dest_fpath, exist_ok=True)
    for file_id in batches[i]:
        src_fpath = f"{fs._local_dir}/{label}/{split}/{feature}/{file_id}.wav"
        if os.path.exists(src_fpath):
            shutil.move(src_fpath, dest_fpath)
        else:
            print(f"File {src_fpath} does not exist, skipping.")

In [117]:
!du -h -d 1 $dest_dir/..

58M	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/train/metadata/batch_0023//../batch_0007


59M	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/train/metadata/batch_0023//../batch_0014
50M	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/train/metadata/batch_0023//../batch_0021
49M	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/train/metadata/batch_0023//../batch_0019
58M	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/train/metadata/batch_0023//../batch_0013
43M	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/train/metadata/batch_0023//../batch_0000
52M	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/train/metadata/batch_0023//../batch_0004
50M	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/train/metadata/batch_0023//../batch_0017
50M	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvised/train/metadata/batch_0023//../batch_0022
60M	/checkpoint/seamless/yaoj/data/batched_seamless_interaction/improvis

In [118]:
label, split = "naturalistic", "train"
feature = "metadata"
file_suffix = ".jsonl"
all_files = fs._fetch_filelist(label=label, split=split)
partitions = fs.partition_filelist(all_files, partition_by="vendor-session-interaction")
batches = fs.create_batches(partitions, batch_size=batch_size_dict[feature])
src_dir = f"{fs._local_dir}{label}/{split}/{feature}"
sub_folders = os.listdir(src_dir)
for i in range(len(batches)):
    dest_dir = f"/checkpoint/seamless/yaoj/data/batched_seamless_interaction/{label}/{split}/{feature}/batch_{i:04d}/"
    os.makedirs(dest_dir, exist_ok=True)
    for subfolder in sub_folders:
        os.makedirs(f"{dest_dir}/{subfolder}", exist_ok=True)
        for file_id in batches[i]:
            src_fpath = f"{src_dir}/{subfolder}/{file_id}{file_suffix}"
            if os.path.exists(src_fpath):
                shutil.move(src_fpath, f"{dest_dir}/{subfolder}/")


In [ ]:
label, split = "improvised", "dev"
feature = "movement"
file_suffix = ".npy"
all_files = fs._fetch_filelist(label=label, split=split)
partitions = fs.partition_filelist(all_files, partition_by="vendor-session-interaction")
batches = fs.create_batches(partitions, batch_size=batch_size_dict[feature])
src_dir = f"{fs._local_dir}{label}/{split}/{feature}"
sub_folders = os.listdir(src_dir)
for i in range(len(batches)):
    dest_dir = f"/checkpoint/seamless/yaoj/data/batched_seamless_interaction/{label}/{split}/{feature}/batch_{i:04d}/"
    os.makedirs(dest_dir, exist_ok=True)
    for subfolder in sub_folders:
        os.makedirs(f"{dest_dir}/{subfolder}", exist_ok=True)
        for file_id in batches[i]:
            src_fpath = f"{src_dir}/{subfolder}/{file_id}{file_suffix}"
            if os.path.exists(src_fpath):
                shutil.move(src_fpath, f"{dest_dir}/{subfolder}/")


['vad', 'transcript']